In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import warnings
warnings.filterwarnings("ignore")

# Keras 가 Tensorflow 를 벡엔드로 사용할 수 있도록 설정합니다.
os.environ["KERAS_BACKEND"] = "tensorflow"

# 실험을 재현하고 동일한 결과를 얻을 수 있는지 확인하기 위해 seed 를 설정합니다.
seed = 2019
np.random.seed(seed)
tf.set_random_seed(seed)

In [38]:
# 데이터셋 로드
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/secom/secom.data"
secom = pd.read_csv(url, header=None, delim_whitespace=True)
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/secom/secom_labels.data"
y = pd.read_csv(url, header=None, usecols=[0], squeeze=True, delim_whitespace=True)
# delim_whitespace = True : 빈 공간(' ')을 구분자로 인식하고 데이터 읽어옴
# squeeze 만약 컬럼 하나만 읽어오면 데이터 구조를 Series로 읽어옴

print('The dataset has {} observations/rows and {} variables/columns.'.format(secom.shape[0], secom.shape[1]))
print('The majority class has {} observations, minority class {}.'.format(y[y == -1].size, y[y == 1].size))
print('The dataset is imbalanced. The ratio of majority class to minority class is {%.2f}:1.' % (float(y[y == -1].size/y[y == 1].size)))

The dataset has 1567 observations/rows and 590 variables/columns.
The majority class has 1463 observations, minority class 104.
The dataset is imbalanced. The ratio of majority class to minority class is {14.07}:1.


In [39]:
dropthese = [i for i in range(590) if secom[i].std() == 0]
secom_categorical = secom.drop(dropthese, axis = 1)

In [40]:
secom_categorical.shape

(1567, 474)

In [41]:
feature_names = secom_categorical.columns
m = list((map(lambda i: sum(secom_categorical[i].isnull()), feature_names)))

In [42]:
print(len(m))
m

474


[6,
 7,
 14,
 14,
 14,
 14,
 9,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 10,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 24,
 24,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 4,
 4,
 4,
 4,
 4,
 7,
 6,
 6,
 6,
 7,
 7,
 7,
 6,
 6,
 6,
 6,
 6,
 794,
 794,
 6,
 24,
 24,
 24,
 24,
 24,
 24,
 24,
 24,
 1,
 12,
 1341,
 0,
 0,
 0,
 51,
 51,
 6,
 2,
 2,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 2,
 2,
 6,
 6,
 6,
 6,
 1018,
 1018,
 1018,
 715,
 0,
 0,
 0,
 0,
 0,
 24,
 0,
 0,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 8,
 8,
 8,
 5,
 6,
 7,
 14,
 14,
 14,
 14,
 9,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 10,
 0,
 1429,
 1429,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 7,
 6,
 6,
 6,
 7,
 7,
 7,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 24,
 24,
 24,
 24,
 24,
 24,
 24,
 24,
 1,
 12,
 1341,
 0,
 0,
 0,
 51,
 51,
 2,
 2,
 2,
 2,
 1018,
 1018,
 1018,
 715,
 0,
 0,
 0,
 0,
 0,
 24,
 0,
 0,
 8,
 8,
 8,
 5,
 6,
 7,
 14,
 

In [43]:
1567*0.6

940.1999999999999

In [45]:
m_940thresh = list(filter(lambda i: (m[i] > 940), range(secom_categorical.shape[1])))
secom_drop_940thresh = secom_categorical.dropna(subset=m_940thresh, axis=1)

KeyError: '[(79, 102, 103, 104, 148, 149, 202, 212, 213, 214, 247, 248, 303, 315, 316, 317, 401, 411, 412, 413, 462, 463, 464, 465)] not in index'

In [8]:
secom_drop_940thresh

,0,2,3,4,6,7,8,9,10,11,...,576,577,582,583,584,585,586,587,588,589
0,3030.93,2187.7333,1411.1265,1.3602,97.6133,0.1242,1.5005,0.0162,-0.0034,0.9455,...,1.6765,14.9509,0.5005,0.0118,0.0035,2.3630,NaN,NaN,NaN,NaN
1,3095.78,2230.4222,1463.6606,0.8294,102.3433,0.1247,1.4966,-0.0005,-0.0148,0.9627,...,1.1065,10.9003,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045
2,2932.61,2186.4111,1698.0172,1.5102,95.4878,0.1241,1.4436,0.0041,0.0013,0.9615,...,2.0952,9.2721,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602
3,2988.72,2199.0333,909.7926,1.3204,104.2367,0.1217,1.4882,-0.0124,-0.0033,0.9629,...,1.7585,8.5831,0.4990,0.0103,0.0025,2.0544,0.0202,0.0149,0.0044,73.8432
4,3032.24,2233.3667,1326.5200,1.5334,100.3967,0.1235,1.5031,-0.0031,-0.0072,0.9569,...,1.6597,10.9698,0.4800,0.4766,0.1045,99.3032,0.0202,0.0149,0.0044,73.8432
5,2946.25,2233.3667,1326.5200,1.5334,100.3967,0.1235,1.5287,0.0167,0.0055,0.9699,...,1.6679,13.7755,0.4949,0.0189,0.0044,3.8276,0.0342,0.0151,0.0052,44.0077
6,3030.27,2230.4222,1463.6606,0.8294,102.3433,0.1247,1.5816,-0.0270,0.0105,0.9591,...,1.1958,8.3645,0.5010,0.0143,0.0042,2.8515,0.0342,0.0151,0.0052,44.0077
7,3058.88,2248.9000,1004.4692,0.7884,106.2400,0.1185,1.5153,0.0157,0.0007,0.9481,...,56.4274,16.0862,0.4984,0.0106,0.0034,2.1261,0.0204,0.0194,0.0063,95.0310
8,2967.68,2248.9000,1004.4692,0.7884,106.2400,0.1185,1.5358,0.0111,-0.0066,0.9494,...,1.3248,14.2892,0.4993,0.0172,0.0046,3.4456,0.0111,0.0124,0.0045,111.6525
9,3016.11,2248.9000,1004.4692,0.7884,106.2400,0.1185,1.5381,0.0159,0.0049,0.9440,...,0.6636,7.4181,0.4967,0.0152,0.0038,3.0687,0.0212,0.0191,0.0073,90.2294


In [33]:
secom_drop_940thresh.shape

(1567, 399)

In [9]:
for column in secom_drop_940thresh:
    mean = secom_drop_940thresh[column].mean()
    secom_drop_940thresh.loc[secom_drop_940thresh[column].isnull(), column] = mean

In [35]:
secom_drop_940thresh

(1567, 399)

In [11]:
from sklearn.decomposition import PCA
# PCA 인스턴스 객체를 생성.
pca = PCA(n_components=35)

In [12]:
pca.fit(secom_drop_940thresh)
secom_pca = pca.transform(secom_drop_940thresh)

In [32]:
pca.explained_variance_ratio_.cumsum()

array([0.59274326, 0.83406358, 0.92559471, 0.94860833, 0.96320481,
       0.96839322, 0.97162054, 0.97476817, 0.97739964, 0.97970553,
       0.98188147, 0.98394928, 0.98564023, 0.9868962 , 0.98810004,
       0.98924119, 0.99035267, 0.99127724, 0.99218204, 0.99304348,
       0.99378244, 0.99450655, 0.99516833, 0.99574108, 0.99629948,
       0.99680377, 0.99729371, 0.99775447, 0.99814858, 0.99851602,
       0.99881227, 0.99897196, 0.99909329, 0.99920953, 0.99930386])

In [13]:
pca.explained_variance_ratio_

array([5.92743256e-01, 2.41320323e-01, 9.15311286e-02, 2.30136226e-02,
       1.45964839e-02, 5.18840763e-03, 3.22731400e-03, 3.14763060e-03,
       2.63147015e-03, 2.30588902e-03, 2.17594614e-03, 2.06781252e-03,
       1.69094928e-03, 1.25596555e-03, 1.20384389e-03, 1.14115037e-03,
       1.11147627e-03, 9.24567330e-04, 9.04801228e-04, 8.61440803e-04,
       7.38959818e-04, 7.24110376e-04, 6.61782482e-04, 5.72744890e-04,
       5.58399309e-04, 5.04291101e-04, 4.89940730e-04, 4.60766172e-04,
       3.94109410e-04, 3.67432776e-04, 2.96249831e-04, 1.59694434e-04,
       1.21334681e-04, 1.16230371e-04, 9.43335335e-05])

In [14]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
sss.get_n_splits(secom_pca, y)

5

In [15]:
for train_index, test_index in sss.split(secom_pca, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = secom_pca[train_index], secom_pca[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print("\n")

TRAIN: [ 932  997 1074 ...  493  648  771] TEST: [ 265  717 1160  740 1237  410 1229  777  853 1200  187  908 1393 1171
  300  786  838  515  439  952  818  923 1017  402  689 1232  108   37
  868  192  620 1187 1234 1467  186  382 1217 1419 1266 1433 1529 1502
  678 1276  142   54   15 1524 1449  714  673 1407  394 1087  549  301
 1432 1352  927 1349 1515 1028 1004 1111 1085  772 1356 1403 1397  841
 1545  825  918    8  879 1162 1188  130 1021  237 1423  712 1305 1001
 1516 1246  147  197  621  198 1115  101 1387  395 1562  677  193  514
  117  757 1226  964 1384 1447  330 1206  271  171   23 1052  984  358
  803  157  231   45 1174  949 1201  904  134 1053  961   72 1417  966
  862  425  205  388  909  600  156  941 1534 1303  445 1138 1306 1044
   39 1401  219 1459 1361  322  120  175  195  726  563 1473  657  807
  930  970 1500  962 1340  209 1131  161  571  481  888 1241  794 1297
  433 1314   78  465  412 1347  836  875 1068 1528  872  661  503  751
 1165 1270   58  643 1294  8

In [16]:
a = 0
b = 0
for i in y_test:
    if i==1:
        a = a+1
    else:
        b= b+1

print(a,b)

31 440


In [17]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=0, k_neighbors=10, sampling_strategy={1:1096,-1:2192})
X_res, y_res = sm.fit_resample(X_train, y_train)

In [18]:
def mlpClassfier(X_train,y_train, epoch):
    from keras.models import Sequential
    from keras.layers import Dense

    model = Sequential()
    model.add(Dense(input_dim=X_train.shape[1],units=60,activation="sigmoid"))
    model.add(Dense(1,activation="tanh"))
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
    model.fit(X_train,y_train,epochs=epoch)

    return model

In [19]:
model = mlpClassfier(X_res,y_res,30)

Using TensorFlow backend.


Epoch 1/30
3288/3288 [==============================] - 3s 996us/step - loss: -3.7365 - acc: 0.2622
Epoch 2/30
3288/3288 [==============================] - 0s 99us/step - loss: -5.6942 - acc: 0.3157
Epoch 3/30
3288/3288 [==============================] - 0s 93us/step - loss: -5.9501 - acc: 0.3896
Epoch 4/30
3288/3288 [==============================] - 0s 97us/step - loss: -5.8266 - acc: 0.5079
Epoch 5/30
3288/3288 [==============================] - 0s 95us/step - loss: -5.8183 - acc: 0.5186
Epoch 6/30
3288/3288 [==============================] - 0s 92us/step - loss: -5.8480 - acc: 0.5240
Epoch 7/30
3288/3288 [==============================] - 0s 96us/step - loss: -5.8537 - acc: 0.5252
Epoch 8/30
3288/3288 [==============================] - 0s 98us/step - loss: -5.8736 - acc: 0.5222
Epoch 9/30
3288/3288 [==============================] - 0s 95us/step - loss: -5.8986 - acc: 0.5222
Epoch 10/30
3288/3288 [==============================] - 0s 96us/step - loss: -5.8568 - acc: 0.5319
Epoch 11

In [20]:
score = model.evaluate(X_test,y_test)
print("%s: %.2f%%" % (model.metrics_names[1], score[1] * 100))
y_predict = model.predict(X_test)
print(y_predict)

471/471 [==============================] - 0s 143us/step
acc: 67.94%
[[-0.7850523 ]
 [ 0.04699877]
 [-0.7559875 ]
 [-0.9165433 ]
 [-0.80780554]
 [-0.51359355]
 [-0.92366606]
 [-0.93437827]
 [-0.96597785]
 [-0.38705102]
 [-0.95511645]
 [-0.7427285 ]
 [-0.7525651 ]
 [-0.96482724]
 [-0.13848427]
 [-0.12870885]
 [-0.9634971 ]
 [-0.9581985 ]
 [ 0.24729125]
 [-0.84939456]
 [-0.30985093]
 [-0.46968022]
 [-0.72555   ]
 [-0.54225004]
 [-0.55493635]
 [-0.7725812 ]
 [ 0.3036836 ]
 [-0.46412566]
 [-0.73684525]
 [-0.7824478 ]
 [-0.7013897 ]
 [-0.7564052 ]
 [ 0.4977237 ]
 [-0.51710546]
 [-0.04714813]
 [-0.69209105]
 [-0.962518  ]
 [-0.7221959 ]
 [-0.3048431 ]
 [-0.9710541 ]
 [-0.5867431 ]
 [-0.9857221 ]
 [-0.5991844 ]
 [-0.95978683]
 [-0.54092216]
 [-0.01532067]
 [-0.57346684]
 [-0.5874845 ]
 [-0.44039088]
 [-0.3868995 ]
 [-0.24009968]
 [-0.8156615 ]
 [-0.7632089 ]
 [-0.48603314]
 [-0.968676  ]
 [-0.9178776 ]
 [-0.69923115]
 [-0.70222044]
 [-0.4106107 ]
 [-0.6436106 ]
 [ 0.19794686]
 [-0.73383725]
 

In [21]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef

In [22]:
def cal_confusion(y, y_test):
    for i,name in enumerate(y):
        if name<0 :
            y[i] = -1
        elif name>=0 :
            y[i] = 1

    print(confusion_matrix(y, y_test))
    print(matthews_corrcoef(y,y_test))

In [23]:
cal_confusion(y_predict, y_test)
a = confusion_matrix(y_predict,y_test)

[[412  27]
 [ 28   4]]
0.064439241520996


In [24]:
tn = a[0][0]
fn = a[0][1]
fp = a[1][0]
tp = a[1][1]

In [27]:
precision = (tp/(tp+fp))*100
recall = (tp/(tp+fn))*100
sensitivity = (tp/(tp+fn))*100
specitivity = (tn/(tn+fp))*100
f_measure = 
acc = ((tp+tn)/(tp+tn+fp+fn))*100

In [31]:
print("acc : %.2f%%, | precision : %.2f%% | tpr : %.2f%% | sensitivity : %.2f%% | specitivity : %.2f%%" %(acc, precision, recall, sensitivity, specitivity))
print(acc, precision, recall, sensitivity, specitivity)

acc : 88.32%, | precision : 12.50% | recall : 12.90% | sensitivity : 12.90% | specitivity : 93.64%
88.32271762208069 12.5 12.903225806451612 12.903225806451612 93.63636363636364
